- Read the dataset from the URL into a dataframe.
- Display the first few rows to make sure it was read properly.

In [2]:
import pandas as pd
# Reading the dataset into a pandas dataframe object
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',
                header=None)
# Print some data to make sure the data was properly read into the dataframe
print df.head(n=3)

         0  1      2      3      4       5        6        7       8   \
0    842302  M  17.99  10.38  122.8  1001.0  0.11840  0.27760  0.3001   
1    842517  M  20.57  17.77  132.9  1326.0  0.08474  0.07864  0.0869   
2  84300903  M  19.69  21.25  130.0  1203.0  0.10960  0.15990  0.1974   

        9    ...        22     23     24      25      26      27      28  \
0  0.14710   ...     25.38  17.33  184.6  2019.0  0.1622  0.6656  0.7119   
1  0.07017   ...     24.99  23.41  158.8  1956.0  0.1238  0.1866  0.2416   
2  0.12790   ...     23.57  25.53  152.5  1709.0  0.1444  0.4245  0.4504   

       29      30       31  
0  0.2654  0.4601  0.11890  
1  0.1860  0.2750  0.08902  
2  0.2430  0.3613  0.08758  

[3 rows x 32 columns]


Let us get a sense of the size of the dataset w

In [1]:
rows, columns = df.shape

         0  1      2      3      4       5        6        7       8   \
0    842302  M  17.99  10.38  122.8  1001.0  0.11840  0.27760  0.3001   
1    842517  M  20.57  17.77  132.9  1326.0  0.08474  0.07864  0.0869   
2  84300903  M  19.69  21.25  130.0  1203.0  0.10960  0.15990  0.1974   

        9    ...        22     23     24      25      26      27      28  \
0  0.14710   ...     25.38  17.33  184.6  2019.0  0.1622  0.6656  0.7119   
1  0.07017   ...     24.99  23.41  158.8  1956.0  0.1238  0.1866  0.2416   
2  0.12790   ...     23.57  25.53  152.5  1709.0  0.1444  0.4245  0.4504   

       29      30       31  
0  0.2654  0.4601  0.11890  
1  0.1860  0.2750  0.08902  
2  0.2430  0.3613  0.08758  

[3 rows x 32 columns]
